# 层类

我们下一个要引入的是**层**（Layer）的概念，作为包裹前向传播通道，和反向传播通道的容器。

我们也将实现第一个层：**线性层**（Linear），用来包裹人工神经元的线性回归功能。

In [7]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [8]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

    def size(self):
        return len(self.data)

    def __str__(self):
        return str(self.data)

### 基础层

基础层是一个抽象类，定义了一个前向传播的接口。

In [10]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

## 数据

### 特征

In [9]:
feature = Tensor([28.1, 58.0])

## 模型

### 线性层

线性层继承了基础层的接口。

人工神经元网络模型包括参数和逻辑两部分。在我们的框架里，这两部将分配到对应的层。而模型就是若干个层的组合。

线性层包装了线性回归的功能，以及对应的参数：权重和偏差。

In [11]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        return Tensor(x.data @ self.weight.data.T + self.bias.data)

## 验证

### 推理

In [12]:
# 建立模型
layer = Linear(feature.size(), 1)
# 模型推理
prediction = layer(feature)

print(f'prediction: {prediction}')

prediction: [43.05]


通过层的概念，我们把模型（参数和逻辑）包装了起来。